In [0]:
!pip install -U tensorflow

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
from tensorflow import keras

In [0]:
import tensorflow_datasets as tfds

In [0]:
print(tf.__version__)

# Dane

Stworzymy model, który będzie rozpoznawał czy dany tekst ma pozytywny czy negatywny wydźwięk (*sentiment analysis*). Korzystać będziemy z kolejnego "standardowego" zbioru danych: recenzje filmów z IMDB.

In [0]:
(train_data, test_data), info = tfds.load(
    # Use the version pre-encoded with an ~8k vocabulary.
    'imdb_reviews/subwords8k', 
    # Return the train/test datasets as a tuple.
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    # Return (example, label) pairs from the dataset (instead of a dictionary).
    as_supervised=True,
    # Also return the `info` structure. 
    with_info=True)

W ten sposób pobraliśmy:

- dane do trenowania `train_data`
- dane testowe `test_data`
- dodatkową strukturę `info`

Structura `info` zawiera w sobie funkcję wykorzystywaną do tłumaczenia (sub-)słów na liczby:

In [0]:
encoder = info.features['text'].encoder

In [0]:
encoder.vocab_size

Tyle sub-słów znajduje się w słowniku.

In [0]:
ehw = encoder.encode("Hello World!")
ehw

In [0]:
encoder.decode(ehw)

In [0]:
for token in ehw:
  print(token, encoder.decode([token]))

#### Zadanie

Zbadaj strukturę zbioru danych do trenowania (`train_data`).


1. Jak zakodowana jest etykietka?
2. W jakim formacie jest recenzje?
3. Wypisz kilka recenzji wraz z etykietką. Zobacz czy ma to sens?

Przyda się funkcja `encoder.decode`.

cf. https://www.tensorflow.org/api_docs/python/tf/data/Dataset


# Model

Zaczniemy od przygotowania danych do uczenia:

In [0]:
type(train_data)

In [0]:
BUFFER_SIZE = 1000

train_batches = (
    train_data
    .shuffle(BUFFER_SIZE)
    .padded_batch(32, train_data.output_shapes))

test_batches = (
    test_data
    .padded_batch(32, train_data.output_shapes))

#### Zadanie

Korzystając z dokumentacji [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) wyjaśnij poszczególne kroki służące przygotowaniu danych. Jak myślisz, jakie jest ich uzasadnienie?

Wyjaśnij rozmiar poniższych batchy:

In [0]:
for example_batch, label_batch in train_batches.take(2):
  print("Batch shape:", example_batch.shape)
  print("label shape:", label_batch.shape)

### Konstrukcja modelu


In [0]:
model = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, 16),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(1, activation='sigmoid')])

model.summary()

`Embedding` jest wartwą, która dokonuje transformacji pojedynczej liczby na rzeczywisty 16-wymiarowy wektor. Jest to w istocie redukcja wymiaru z `encoder.vocab_size` (czyli 8000) do 16.

[`GlobalAveragePooling1D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D) wylicza średnią wartość z poprzedniej warstwy (zwróć uwagę na zmianę kształtu).

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
history = model.fit(train_batches,
                    epochs=10,
                    validation_data=test_batches,
                    validation_steps=30)

In [0]:
loss, accuracy = model.evaluate(test_batches)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

### Wykres parametrów uczenia

Wizualizacja graficzna parametrów uczenia bywa przydatna.

Przeanalizuj poniższy kod:

In [0]:
import matplotlib.pyplot as plt

history_dict = history.history
history_dict.keys()

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [0]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

#### Zadanie

Przeprowadź trenowanie dla większej liczby epok (np. 30, 50), narysuj jeszcze raz wykresy. Co możesz powiedzieć o overfittingu?

#### Zadanie

Wykonaj eksperymenty z inną architekturą:

- dodaj więcej warstw ukrytych o różnych szerokościach
- zmień wymiar embeddingu

Spróbuj zwiększyć dokładność modelu.